# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key


# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

In [ ]:
cities[:10]

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
url = "http://api.openweathermap.org/data/2.5/weather?"
city = "sauda"
query_url = url + "appid=" + weather_api_key + "&q=" + city
response = requests.get(query_url).json()
response

In [ ]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

# set up lists to hold reponse info
lat = []
temp = []
long = []
cloudiness=[]
humidity = []
windspeed = []
country=[]
date=[]
city_names=[]


# Loop through the list of cities and perform a request for data on each
for city in cities:
    try:
        response = requests.get(query_url + city).json()
        lat.append(response['coord']['lat'])
        temp.append(response['main']['temp_max'])
        long.append(response['coord']['lon'])
        cloudiness.append(response['clouds']['all'])
        humidity.append(response['main']['humidity'])
        windspeed.append(response['wind']['speed'])
        country.append(response['sys']['country'])
        date.append(response['dt'])
        city_names.append(city)
        print (city)
        
    except:
        print("City Not Found")

# print(f"The latitude information received is: {lat}")
# print(f"The temperature information received is: {temp}")

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
city_df=pd.DataFrame({"City":city_names,"Lat":lat, "Lng":long,
                      "Max Temp":temp, "Humidity":humidity,
                      "Cloudiness":cloudiness,"Wind Speed":windspeed,
                      "Country":country,
                     "Date":date})
city_df.head()

In [ ]:
# Export the data to csv
city_df.to_csv(r'../output_data/weather_data.csv')

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
# Display a statistical overview of the DataFrame.
city_df.describe()
#there is no value greater than 100. Max value is 100 

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
plt.scatter(city_df['Lat'],city_df['Max Temp'],facecolors = "red", edgecolors ="black", marker ="o")
plt.title(f'City Latitude vs.Temperature ({time.strftime("%m/%d/%Y")})')
plt.xlabel("Latitude")
plt.ylabel("Temperature(F)")
plt.grid(linestyle='-', linewidth=1,alpha = 0.5)

# Save Figure
plt.savefig("../output_data/City_Latitude_vs_Temperature.png")

plt.show()

## Latitude vs. Humidity Plot

In [ ]:
# Latitude vs. Humidity Scatter Plot 
plt.scatter(city_df["Lat"],city_df["Humidity"],facecolors = "blue", edgecolors ="black",marker ="o")
plt.title(f'City Latitude vs. Humidity ({time.strftime("%m/%d/%Y")})')
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.grid(linestyle='-', linewidth=1, alpha = 0.5)

# Save Figure
plt.savefig("../output_data/City_Latitude_vs_Humidity.png")

plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
# Latitude vs. Cloudiness Scatter Plot
plt.scatter(city_df['Lat'],city_df['Cloudiness'],facecolors = "red", edgecolors ="black", marker ="o")
plt.title(f'City Latitude vs. Cloudiness ({time.strftime("%m/%d/%Y")})')
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid(linestyle='-', linewidth=1, alpha = 0.5)

# Save Figure
plt.savefig("../output_data/City_Latitude_vs_Cloudiness.png")

plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
# Latitude vs. Wind Speed Scatter Plot 
plt.scatter(city_df["Lat"],city_df["Wind Speed"],facecolors = "steelblue", edgecolors ="black", marker ="o")
plt.title(f'City Latitude vs. Wind Speed ({time.strftime("%m/%d/%Y")})')
plt.xlabel("Latitude")
plt.ylabel("WindSpeed")
plt.grid(linestyle='-', linewidth=1, alpha = 0.5)

# Save Figure
plt.savefig("../output_data/City_Latitude_vs_WindSpeed.png")

plt.show()

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames
#northern Hemisphere Data frame 
N_df=city_df[city_df['Lat']>=0]


In [ ]:
#Southern Hemisphere Data frame 
S_df=city_df[city_df['Lat']<0]
S_df

In [ ]:
# Print out the r-squared value along with the plot.
def linearplt(x_values,y_values,ytitle,coordinates):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r-")
    plt.annotate(line_eq,coordinates,fontsize=15,color="red")
    plt.xlabel("Latitude")
    plt.ylabel(ytitle)
    print(f"The r-squared is: {rvalue**2}")
    plt.show()

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values=N_df['Lat']
y_values=N_df['Max Temp']
linearplt(x_values,y_values,"Max Temp",(10,-10))

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values=S_df['Lat']
y_values=S_df['Max Temp']
linearplt(x_values,y_values,"Max Temp",(10,-10))

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values=N_df['Lat']
y_values=N_df['Humidity']
linearplt(x_values,y_values,"Max Temp",(10,-10))

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values=S_df['Lat']
y_values=S_df['Humidity']
linearplt(x_values,y_values,"Max Temp",(10,-10))

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values=N_df['Lat']
y_values=N_df['Cloudiness']
linearplt(x_values,y_values,"Max Temp",(10,-10))

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values=S_df['Lat']
y_values=S_df['Cloudiness']
linearplt(x_values,y_values,"Max Temp",(10,-10))

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values=N_df['Lat']
y_values=N_df['Wind Speed']
linearplt(x_values,y_values,"Max Temp",(10,-10))

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values=S_df['Lat']
y_values=S_df['Wind Speed']
linearplt(x_values,y_values,"Max Temp",(10,-10))